In [1]:
import boto3
import pandas as pd
import numpy as np
import psycopg2
import configparser

## Leemos archivo de configuración


In [2]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

## Nos identificamos con AWS


In [3]:
aws_rds_connection = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                              aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                              region_name='us-east-2')

## Verificamos instancias de AWS disponibles en el usuario


In [4]:
rds_instances_ids = []

aws_response = aws_rds_connection.describe_db_instances()

for response in aws_response['DBInstances']:
    rds_instances_ids.append(response['DBInstanceIdentifier'])

print(f"Instancias Disponibles: {rds_instances_ids}")

Instancias Disponibles: ['tienda-transactional']


## Creamos instancia de base de datos en AWS - PostGres


In [5]:
try:
    response = aws_rds_connection.create_db_instance(
        DBInstanceIdentifier=config.get('TRANSACC', 'DB_INSTANCE_ID'),
        DBName=config.get('TRANSACC', 'DB_NAME'),
        DBInstanceClass='db.t3.micro',
        Engine='postgres',
        MasterUsername=config.get('TRANSACC', 'DB_USER'),
        MasterUserPassword=config.get('TRANSACC', 'DB_PASSWORD'),
        Port=int(config.get('TRANSACC', 'DB_PORT')),
        PubliclyAccessible=True,
        VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
        AllocatedStorage=15
    )
    print(response)
except aws_rds_connection.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La instancia ya existe")
except Exception as ex:
    print("Error!", ex)

La instancia ya existe


## Obtenemos el hostname de la instancia


In [6]:
try:
    instance = aws_rds_connection.describe_db_instances(DBInstanceIdentifier=config.get('TRANSACC', 'DB_INSTANCE_ID'))
    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)
except Exception as ex:
    print("Error!!", ex)

tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com


## Nos conectamos a la base de datos desde Python


In [7]:
import sql_queries
#sql_queries.DDL_QUERY

In [8]:
try:
    db_pg_conn = psycopg2.connect(
        database=config.get('TRANSACC', 'DB_NAME'),
        user=config.get('TRANSACC', 'DB_USER'),
        password=config.get('TRANSACC', 'DB_PASSWORD'),
        host=RDS_HOSTNAME,
        port=int(config.get('TRANSACC', 'DB_PORT'))
    )
    cursor = db_pg_conn.cursor()
    cursor.execute(sql_queries.DDL_QUERY)
    db_pg_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exception as ex:
    print("Error!", ex)

Base de Datos Creada Exitosamente


## Insertamos datos en la base de datos

In [9]:
def insertData2SQL(data_dict, table_name, driver):

    df_data = pd.DataFrame.from_records(data_dict)
    try:
        response = df_data.to_sql(table_name, driver, index=False, if_exists='append')
        print(f"Se han insertado {response} nuevos registros")
    except Exception as ex:
        print(ex)

In [10]:
def insertData2SQL_categoria(data_dict, table_name, driver):
    df_data = pd.DataFrame.from_records(data_dict)
    try:
        for index, row in df_data.iterrows():
            estado_value = "'1'" if row.to_dict()['estado'] else "'0'"
            query = f"INSERT INTO {table_name} VALUES ({row.to_dict()['idcategoria']}, '{row.to_dict()['nombre']}', '{row.to_dict()['descripcion']}', CAST({estado_value} AS BIT))"
            cursor.execute(query)
        db_pg_conn.commit()
        print(f"Se han insertado {len(df_data)} nuevos registros")
    except Exception as ex:
        print(ex)

In [11]:
def insertData2SQL_rol(data_dict, table_name, driver):
    df_data = pd.DataFrame.from_records(data_dict)
    try:
        for index, row in df_data.iterrows():
            estado_value = "'1'" if row.to_dict()['estado'] else "'0'"
            query = f"INSERT INTO {table_name} VALUES ({row.to_dict()['idrol']}, '{row.to_dict()['nombre']}', '{row.to_dict()['descripcion']}', CAST({estado_value} AS BIT))"
            cursor.execute(query)
        db_pg_conn.commit()
        print(f"Se han insertado {len(df_data)} nuevos registros en la tabla {table_name}")
    except Exception as ex:
        print(ex)

In [12]:
def insertData2SQL_articulo(data_dict, table_name, driver):
    df_data = pd.DataFrame.from_records(data_dict)
    try:
        for index, row in df_data.iterrows():
            estado_value = "'1'" if row.to_dict()['estado'] else "'0'"
            query = f"INSERT INTO {table_name} VALUES ({row.to_dict()['idarticulo']}, '{row.to_dict()['idcategoria']}', '{row.to_dict()['codigo']}', '{row.to_dict()['nombre']}', '{row.to_dict()['precio_venta']}', '{row.to_dict()['stock']}', '{row.to_dict()['descripcion']}', '{row.to_dict()['imagen']}', CAST({estado_value} AS BIT))"
            cursor.execute(query)
        db_pg_conn.commit()
        print(f"Se han insertado {len(df_data)} nuevos registros en la tabla {table_name}")
    except Exception as ex:
        print(ex)

In [13]:
import binascii

def insertData2SQL_usuario(data_dict, table_name, driver):
    try:
        for row in data_dict:
            estado_value = 1 if row['estado'] else 0
            query = f"INSERT INTO {table_name} VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, CAST(%s AS BIT))"
            cursor.execute(query, (
                row['idusuario'],
                row['idrol'],
                row['nombre'],
                row['tipo_documento'],
                row['num_documento'],
                row['direccion'],
                row['telefono'],
                row['email'],
                binascii.hexlify(row['clave']).decode('utf-8'),
                estado_value
            ))
        db_pg_conn.commit()
        print(f"Se han insertado {len(data_dict)} nuevos registros en la tabla {table_name}")
    except Exception as ex:
        print(ex)



#def insertData2SQL_usuario(data_dict, table_name, cursor):
#    try:
#        for index, row in enumerate(data_dict, 1):
#            estado_value = 1 if row['estado'] else 0
#            # Convertir datos binarios a representación hexadecimal
#            clave_hex = binascii.hexlify(row['clave']).decode('utf-8')
#            query = f"INSERT INTO {table_name} VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
#            cursor.execute(query, (row['idusuario'], row['idrol'], row['nombre'], row['tipo_documento'],
#                                   row['num_documento'], row['direccion'], row['telefono'], row['email'],
#                                   clave_hex, estado_value))
#        print(f"Se han insertado {index} nuevos registros en la tabla {table_name}")
#    except psycopg2.Error as e:
#        print("Error al insertar datos:", e)

In [14]:
def generate_unique_random_numbers(start, end, count):
    unique_numbers = set()
    while len(unique_numbers) < count:
        random_number = np.random.randint(start, end)
        unique_numbers.add(random_number)
    return list(unique_numbers)



In [15]:
# Define una función para generar un número de teléfono con un número específico de dígitos
def generate_phone_number(digits):
    # Utiliza el formato específico para el número de teléfono, ajustando el número de dígitos según sea necesario
    phone_number = fake.phone_number()[:digits]
    return phone_number

In [16]:
driver = f"""postgresql://{config.get('TRANSACC', 'DB_USER')}:{config.get('TRANSACC', 'DB_PASSWORD')}@{RDS_HOSTNAME}:{config.get('TRANSACC', 'DB_PORT')}/{config.get('TRANSACC', 'DB_NAME')}"""

# Agregando datos a la tabla categoria

In [17]:
data_categoria = [
    {'idcategoria': 39582, 'nombre': 'Ropa y Accesorios', 'descripcion': 'Incluye prendas de vestir para hombres, mujeres y ninos, asi como accesorios como bolsos, sombreros, bufandas, etc.', 'estado': True},
    {'idcategoria': 82016, 'nombre': 'Calzado', 'descripcion': 'Incluye zapatos formales, tenis, botas, sandalias, para hombres, mujeres y ninos.', 'estado': True},
    {'idcategoria': 51470, 'nombre': 'Juguetes y Juegos', 'descripcion': 'Una amplia gama de juguetes para ninos de todas las edades, asi como juegos de mesa y articulos deportivos.', 'estado': True},
    {'idcategoria': 75369, 'nombre': 'Electronicos', 'descripcion': 'Desde telefonos moviles, computadoras portatiles y tabletas hasta accesorios como auriculares, cargadores y fundas.', 'estado': True},
    {'idcategoria': 20934, 'nombre': 'Mascotas', 'descripcion': 'Ofrece productos para el cuidado de mascotas, alimentos, juguetes y accesorios para perros, gatos y otras mascotas.', 'estado': True},
    {'idcategoria': 64218, 'nombre': 'Alimentos y Bebidas', 'descripcion': 'Desde alimentos enlatados y productos secos hasta bebidas como vinos, cervezas artesanales y licores.', 'estado': True},
    {'idcategoria': 13750, 'nombre': 'Libros y Papeleria', 'descripcion': 'Ofrece una selección de libros para todas las edades y generos, asi como suministros de papeleria como cuadernos, bolígrafos y lapices.', 'estado': True},
    {'idcategoria': 40879, 'nombre': 'Belleza y Cuidado Personal', 'descripcion': 'Incluye productos de cuidado de la piel, maquillaje, perfumes, asi como herramientas y accesorios para el cuidado del cabello.', 'estado': True},
    {'idcategoria': 97521, 'nombre': 'Joyeria', 'descripcion': 'Desde anillos, pulseras y collares hasta relojes y joyeria personalizada.', 'estado': True},
    {'idcategoria': 32684, 'nombre': 'Hogar y Decoracion', 'descripcion': 'Ofrece muebles para el hogar, articulos de decoracion como cuadros y jarrones, asi como utensilios de cocina y electrodomesticos.', 'estado': True}
]

# Insertamos data en la tabla categoria
insertData2SQL_categoria(data_categoria, 'categoria', driver)

Se han insertado 10 nuevos registros


# Agregando datos a la tabla rol


In [18]:
data_roles = [
    {'idrol': 10001, 'nombre': 'Administrador', 'descripcion': 'Este rol tiene acceso completo al sistema y puede realizar todas las acciones.', 'estado': True},
    {'idrol': 10002, 'nombre': 'Moderador', 'descripcion': 'Este rol tiene permisos para moderar contenido y realizar acciones específicas.', 'estado': True},
    {'idrol': 10003, 'nombre': 'Usuario Estándar', 'descripcion': 'Este rol tiene acceso limitado y puede realizar acciones básicas en el sistema.', 'estado': True},
    {'idrol': 10004, 'nombre': 'Soporte Técnico', 'descripcion': 'Este rol proporciona soporte técnico y tiene acceso a herramientas de diagnóstico.', 'estado': True},
    {'idrol': 10005, 'nombre': 'Invitado', 'descripcion': 'Este rol tiene acceso limitado y solo puede ver contenido público.', 'estado': True}
]

# Insertamos data en la tabla rol
insertData2SQL_rol(data_roles, 'rol', driver)

Se han insertado 5 nuevos registros en la tabla rol


# Agregando datos a la tabla articulo

In [19]:
from faker import Faker
import random

fake = Faker()

# Lista de palabras clave que representan tipos comunes de artículos
palabras_clave = ["Camisa", "Pantalón", "Zapatos", "Tenis", "Vestido", "Reloj", "Teléfono", "Portátil", "Tableta", "Juguete", "Libro", "Silla", "Mesa", "Lámpara", "Cuadro", "Cocina", "Cama", "Alfombra", "Jarrón", "Perfume", "Maquillaje", "Galleta", "Pelota", "Anillo", "Suéter", "Bolígrafo"]

nombres_articulos = []

# Generar nombres de artículos aleatorios combinando palabras aleatorias con palabras clave
for _ in range(1000):
    palabra_aleatoria = fake.word()
    nombre_articulo = random.choice(palabras_clave) + " " + palabra_aleatoria.capitalize()
    nombres_articulos.append(nombre_articulo)

print(nombres_articulos[:100])  # Imprimir los primeros 10 nombres de artículos generados


['Camisa Since', 'Pelota Hour', 'Pelota Just', 'Portátil Force', 'Juguete Style', 'Lámpara Control', 'Pantalón According', 'Camisa Billion', 'Bolígrafo Indicate', 'Pantalón Business', 'Bolígrafo Memory', 'Alfombra Lot', 'Suéter Western', 'Perfume Ready', 'Bolígrafo Policy', 'Jarrón Present', 'Perfume Ok', 'Cocina Subject', 'Zapatos Get', 'Reloj No', 'Alfombra Focus', 'Bolígrafo Half', 'Cuadro Decision', 'Lámpara Parent', 'Maquillaje Area', 'Pantalón Wife', 'Silla Out', 'Reloj Rate', 'Reloj Audience', 'Cocina All', 'Vestido World', 'Reloj Type', 'Anillo Moment', 'Cuadro Speak', 'Bolígrafo Water', 'Anillo Gun', 'Galleta Seek', 'Cocina Especially', 'Alfombra Time', 'Teléfono Wall', 'Lámpara Break', 'Silla Never', 'Suéter Partner', 'Cama Better', 'Bolígrafo Business', 'Suéter Police', 'Portátil Pretty', 'Pantalón Final', 'Silla Unit', 'Pantalón Resource', 'Pelota Ready', 'Pelota Miss', 'Maquillaje Main', 'Pantalón Alone', 'Anillo Us', 'Cocina Message', 'Jarrón Student', 'Mesa Religious', '

In [20]:
def generar_codigo(length):
    caracteres = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
    codigo = ''.join(random.choice(caracteres) for _ in range(length))
    return codigo

In [21]:
len(nombres_articulos)

1000

In [22]:
cantidad_articulos = 1000

data_articulos = []

start_number_articulo = 100000
end_number_articulo = 999999
number_count_articulo = cantidad_articulos

unique_random_numbers_idarticulos = generate_unique_random_numbers(start_number_articulo, end_number_articulo, number_count_articulo)

for index, articulo in enumerate(range(cantidad_articulos)):
    nuevo_articulo = {
        'idarticulo': unique_random_numbers_idarticulos[index],
        'idcategoria': random.sample(data_categoria, 1)[0]['idcategoria'],
        'codigo': generar_codigo(8),
        'nombre': nombres_articulos[index],
        'precio_venta': round(random.uniform(10, 1000),2),
        'stock': np.random.randint(1, 100),
        'descripcion': f'Artículo {nombres_articulos[index]} en venta en tienda',
        'imagen': 'fig_' + str(index),
        'estado': True
    }
    data_articulos.append(nuevo_articulo)
insertData2SQL_articulo(data_articulos, 'articulo', driver)

Se han insertado 1000 nuevos registros en la tabla articulo


## Agregando datos a la tabla persona


In [23]:
tipo_persona = ['cliente', 'proveedor']
tipo_documento = ['DNI', 'NIE', 'Pasaporte', 'Tarjeta Residencia']

cantidad_personas = 1000

start_number_persona = 100000000
end_number_persona = 999999999
number_count_persona = cantidad_personas

unique_random_numbers_idpersona = generate_unique_random_numbers(start_number_persona, end_number_persona, number_count_persona)

start_number_num_doc_persona = 100000000000000
end_number_num_doc_persona = 199999999999999
number_count_num_doc_persona = cantidad_personas

unique_random_numbers_num_doc_persona = generate_unique_random_numbers(start_number_num_doc_persona, end_number_num_doc_persona, number_count_num_doc_persona)

data_personas = []
for index, persona in enumerate(range(cantidad_personas)):
    perfil_persona = fake.profile()
    nombre_persona = perfil_persona['name']
    direccion_persona = perfil_persona['address']
    direccion_persona = direccion_persona.replace("\n", " ")
    email_persona = perfil_persona['mail']
    # Generar un número de teléfono
    numero_telefono = generate_phone_number(8)
    nueva_persona = {
        'idpersona': unique_random_numbers_idpersona[index],
        'tipo_persona': tipo_persona[np.random.randint(0,2)],
        'nombre': nombre_persona,
        'tipo_documento': tipo_documento[np.random.randint(0,4)],
        'num_documento': unique_random_numbers_num_doc_persona[index],
        'direccion': direccion_persona,
        'telefono': numero_telefono,
        'email': email_persona,
    }
    data_personas.append(nueva_persona)
insertData2SQL(data_personas, 'persona', driver)

Se han insertado 1000 nuevos registros


## Agregando datos a la tabla usuario


In [24]:
import os

cantidad_usuarios = 1000
estados = [True, False]

start_number_usuario = 1000000
end_number_usuario = 9999999
number_count_usuario = cantidad_usuarios

unique_random_numbers_idusuario = generate_unique_random_numbers(start_number_usuario, end_number_usuario, number_count_usuario)

start_number_num_doc_usuario = 100000000000000
end_number_num_doc_usuario = 199999999999999
number_count_num_doc_usuario = cantidad_usuarios

unique_random_numbers_num_doc_usuario = generate_unique_random_numbers(start_number_num_doc_usuario, end_number_num_doc_usuario, number_count_num_doc_usuario)

data_usuarios = []
for index, usuario in enumerate(range(cantidad_usuarios)):
    perfil_persona = fake.profile()
    nombre_persona = perfil_persona['name']
    direccion_persona = perfil_persona['address']
    direccion_persona = direccion_persona.replace("\n", " ")
    email_persona = perfil_persona['mail']
    longitud_bytes = 10
    dato_binario = os.urandom(longitud_bytes)
    # Generar un número de teléfono
    numero_telefono = generate_phone_number(8)
    
    nuevo_usuario = {
        'idusuario': unique_random_numbers_idusuario[index],
        'idrol': random.choices(data_roles, weights=(0.1, 0.2, 0.4, 0.1, 0.2), k=1)[0]['idrol'],
        'nombre': nombre_persona,
        'tipo_documento': tipo_documento[np.random.randint(0,3)],
        'num_documento': unique_random_numbers_num_doc_usuario[index],
        'direccion': direccion_persona,
        'telefono': numero_telefono,
        'email': email_persona,
        'clave': dato_binario,
        'estado': estados[np.random.randint(0,2)],
    }
    data_usuarios.append(nuevo_usuario)
insertData2SQL_usuario(data_usuarios, 'usuario', driver)

Se han insertado 1000 nuevos registros en la tabla usuario


## Agregando datos a la tabla venta


In [25]:
from datetime import datetime, timedelta

cantidad_venta = 1000
tipos_comprobante = ['Factura', 'Boleta de Venta', 'Ticket de Venta', 'Nota de Crédito', 'Nota de Débito', 'Recibo']
estados_venta = ['Pendiente', 'En proceso', 'Completada', 'Cancelada', 'Reembolsada']
# Definir el rango de fechas
fecha_inicio = datetime(2015, 1, 1)
fecha_fin = datetime(2023, 12, 31)

# Generar una fecha aleatoria dentro del rango
fecha_aleatoria = fecha_inicio + timedelta(days=random.randint(0, (fecha_fin - fecha_inicio).days))

start_number_venta = 111111
end_number_venta = 999999
number_count_venta = cantidad_venta

unique_random_numbers_idventa = generate_unique_random_numbers(start_number_venta, end_number_venta, number_count_venta)

start_number_serie_comprob = 1000000
end_number_serie_comprob = 9999999
number_count_serie_comprob = cantidad_venta

unique_random_numbers_serie_comprob = generate_unique_random_numbers(start_number_serie_comprob, end_number_serie_comprob, number_count_serie_comprob)

start_number_num_comprob = 10000000
end_number_num_comprob = 99999999
number_count_num_comprob = cantidad_venta

unique_random_numbers_num_comprob = generate_unique_random_numbers(start_number_num_comprob, end_number_num_comprob, number_count_num_comprob)

data_venta = []
for index, venta in enumerate(range(cantidad_venta)):
    index_usuario = np.random.randint(0, len(data_usuarios))  # Generar un índice aleatorio
    idusuario = data_usuarios[index_usuario]['idusuario']  # Obtener el idusuario correspondiente al índice
    # Generar una fecha aleatoria dentro del rango
    fecha_aleatoria = fecha_inicio + timedelta(days=random.randint(0, (fecha_fin - fecha_inicio).days))
    nueva_venta = {
        'idventa': unique_random_numbers_idventa[index],
        'idcliente': random.sample(data_personas, 1)[0]['idpersona'],
        'idusuario': random.sample(data_usuarios, 1)[0]['idusuario'],
        'tipo_comprobante': tipos_comprobante[np.random.randint(0,6)],
        'serie_comprobante': unique_random_numbers_serie_comprob[index],
        'num_comprobante': unique_random_numbers_num_comprob[index],
        'fecha': fecha_aleatoria,
        'impuesto': round(random.uniform(0, 99.99), 2),
        'total': round(random.uniform(0, 99999.99), 2),
        'estado': estados_venta[np.random.randint(0,5)]
    }
    data_venta.append(nueva_venta)
insertData2SQL(data_venta, 'venta', driver)

Se han insertado 1000 nuevos registros


## Agregando datos a la tabla detalles_venta


In [26]:
cantidad_detalles_venta = 5000

start_number_detalle_venta = 3333333
end_number_detalle_venta = 9999999
number_count_detalle_venta = cantidad_detalles_venta

unique_random_numbers_iddetalle_venta = generate_unique_random_numbers(start_number_detalle_venta, end_number_detalle_venta, number_count_detalle_venta)

data_detalles_venta = []
for index, detalles_venta in enumerate(range(cantidad_detalles_venta)):
    muestra_tabla_articulo = random.sample(data_articulos, 1)
    idarticulo_muestra = muestra_tabla_articulo[0]['idarticulo']
    precio_muestra = muestra_tabla_articulo[0]['precio_venta']
    nuevo_detalle_venta = {
        'iddetalle_venta': unique_random_numbers_iddetalle_venta[index],
        'idventa': random.sample(data_venta, 1)[0]['idventa'],
        'idarticulo': idarticulo_muestra,
        'cantidad': np.random.randint(0,100),
        'precio': precio_muestra,
        'descuento': round(random.uniform(0, 999.99), 2)
    }
    data_detalles_venta.append(nuevo_detalle_venta)
insertData2SQL(data_detalles_venta, 'detalle_venta', driver)

Se han insertado 1000 nuevos registros


## Agregando datos  a la tabla ingreso


In [27]:
cantidad_ingresos = 1000
estados_ingreso = ['Pendiente', 'En proceso', 'Completada', 'Cancelada', 'Reembolsada']

start_number_ingreso = 100000000
end_number_ingreso = 999999999
number_count_ingreso = cantidad_ingresos

unique_random_numbers_idingreso = generate_unique_random_numbers(start_number_ingreso, end_number_ingreso, number_count_ingreso)

start_number_serie_comprob_ingreso = 1000000
end_number_serie_comprob_ingreso = 9999999
number_count_serie_comprob_ingreso = cantidad_ingresos

unique_random_numbers_serie_comprob_ingreso = generate_unique_random_numbers(start_number_serie_comprob_ingreso, end_number_serie_comprob_ingreso, number_count_serie_comprob_ingreso)

data_ingresos = []
for index, ingreso in enumerate(range(cantidad_ingresos)):
    fecha_aleatoria = fecha_inicio + timedelta(days=random.randint(0, (fecha_fin - fecha_inicio).days))
    nuevo_ingreso = {
        'idingreso': unique_random_numbers_idingreso[index],
        'idproveedor': random.sample(data_personas, 1)[0]['idpersona'],
        'idusuario': random.sample(data_usuarios, 1)[0]['idusuario'],
        'tipo_comprobante':  tipos_comprobante[np.random.randint(0,6)],
        'serie_comprobante': unique_random_numbers_serie_comprob_ingreso[index],
        'fecha': fecha_aleatoria,
        'impuesto': round(random.uniform(0, 99.99), 2),
        'total': round(random.uniform(0, 99999.99), 2),
        'estado': estados_ingreso[np.random.randint(0,5)]
    }
    data_ingresos.append(nuevo_ingreso)
insertData2SQL(data_ingresos, 'ingreso', driver)

Se han insertado 1000 nuevos registros


## Agregando datos a la tabla detalle_ingreso


In [28]:
cantidad_detalles_ingreso = 1000

start_number_detalle_ingreso = 100000000
end_number_detalle_ingreso = 999999999
number_count_detalle_ingreso = cantidad_detalles_ingreso

unique_random_numbers_iddetalle_ingreso = generate_unique_random_numbers(start_number_detalle_ingreso, end_number_detalle_ingreso, number_count_detalle_ingreso)

data_detalles_ingreso = []
for index, detalles_ingreso in enumerate(range(cantidad_detalles_ingreso)):
    muestra_tabla_articulo = random.sample(data_articulos, 1)
    idarticulo_muestra = muestra_tabla_articulo[0]['idarticulo']
    precio_muestra = muestra_tabla_articulo[0]['precio_venta']
    nuevo_detalle_ingreso = {
        'iddetalle_ingreso': unique_random_numbers_iddetalle_ingreso[index],
        'idingreso': random.sample(data_ingresos, 1)[0]['idingreso'],
        'idarticulo': idarticulo_muestra,
        'cantidad': np.random.randint(0,100),
        'precio': precio_muestra
    }
    data_detalles_ingreso.append(nuevo_detalle_ingreso)
insertData2SQL(data_detalles_ingreso, 'detalle_ingreso', driver)

Se han insertado 1000 nuevos registros


In [29]:
# Consulta para obtener el nombre de todas las tablas en el esquema público
consulta_tablas = """
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public' 
AND table_type = 'BASE TABLE';
"""
# Ejecutar la consulta
cursor.execute(consulta_tablas)
# Obtener los nombres de las tablas
tablas = cursor.fetchall()

In [30]:
# Iterar sobre las tablas y eliminarlas una por una
for tabla in tablas:
    eliminar_tabla = f"DROP TABLE IF EXISTS {tabla[0]} CASCADE;"
    cursor.execute(eliminar_tabla)

# Confirmar los cambios y cerrar la conexión
db_pg_conn.commit()
db_pg_conn.close()